In [1]:
!pip install openai
!pip install requests


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import csv
import os
from googletrans import Translator
from datetime import datetime
import codecs
import openai
import requests
import textwrap
import re
import time

In [2]:
def remove_intro(sentence):
    return re.sub(r'^(.*?我.*?\s|谢谢。\s*)', '', sentence)

In [3]:
# Translate the title
def translate_title(url):
    translated_title = openai.Completion.create(
        engine="text-davinci-002", #enough for translation, faster speed
        prompt=f"Here is the title for a piece of news: \n{url}\n Please translate it to Chinese",
        max_tokens=100, n=1, stop=None, temperature=0.5,)
    # set temperature higher for creative results
    translated_title_text = translated_title.choices[0].text.strip()
    translated_title_text = re.sub(r"^[^\w\d]+", "", translated_title_text)
    
    return translated_title_text

# Provide a Summary in Chinese
def summarize_and_translate(url):
    summary = openai.Completion.create(
        engine="text-davinci-003", #3 has better performance in summarizing
        prompt=f"Here is the url for a piece of news: \n{url}\nPlease summarize it to a paragraph of Chinese within 100 word limit. If you can't access the content, please output <can't access>"
        max_tokens=1000, n=1, stop=None, temperature=0.5,)
    
    summary_text = summary.choices[0].text.strip()
    summary_text = re.sub(r"^[^\w\d]+", "", summary_text)

    return summary_text

def main(df):
    wrapper = textwrap.TextWrapper(width=80)
    for i in range(len(df)):      
        title = translate_title(df['Headline'].iloc[i])
        df['Headline'].iloc[i] =  df['Headline'].iloc[i] + ' ' + wrapper.fill(title)
        
        summary = summarize_and_translate(df['URL'].iloc[i])
        
        tmp = re.sub(r'^[^\w]*', '', df['Hit Sentence'].iloc[i])
        tmp = tmp[0].upper() + tmp[1:]
        df['Hit Sentence'].iloc[i] = tmp + ' ' + remove_intro(wrapper.fill(summary))

        print(i,"\nTitle in simplified Chinese:")
        print(wrapper.fill(title))

        print("\nSummary in simplified Chinese:")
        print(wrapper.fill(summary))

In [4]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.width', None)  # Auto-detect the screen width
pd.set_option('display.max_colwidth', None) 

In [5]:
df1 = pd.read_csv("~/Downloads/DataSource1 - Nov 29, 2023 - 4 49 28 PM.csv", sep="\t", encoding="UTF-16")
df2 = pd.read_csv("~/Downloads/DataSource2 - Nov 29, 2023 - 4 48 57 PM.csv", sep="\t", encoding="UTF-16")

In [6]:
df = pd.concat([df1, df2])

In [7]:
df.shape

(901, 49)

In [8]:
# drop duplicate columns before translation to save time
df = df.drop_duplicates(subset=['Hit Sentence'], ignore_index = True)
df = df.drop_duplicates(subset=['Headline'], ignore_index = True)

In [9]:
df.shape

(506, 49)

In [1]:
# below is to format the file to the output I need
def convert_date_format(date_str):
    dt_object = datetime.strptime(date_str, "%d-%b-%Y %I:%M%p")
    return dt_object.strftime('%Y/%m/%d')

df['Date'] = df['Date'].apply(convert_date_format)

NameError: name 'df' is not defined

In [11]:
today = datetime.today()
# Format the date (e.g., '2023-08-10')
final_name = today.strftime('%Y-%m-%d')
df['作者'] = ''

In [12]:
# create the file for google translate
cols = df[['Headline', 'Hit Sentence']]
cols['index'] = cols.index
cols.to_excel(f'~/Downloads/translate{final_name}.xlsx', index=False)

/Users/wjq/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# ****google translate here + select news needs + save translated file

In [13]:
selected_urls = df.iloc[[
1,
2,
37,
38,
45,
47,
53,
162,
184,
225,
263,
395,
413,
416,
440
], [1,4,2,5,7,39,0]]

In [14]:
%%time
openai.api_key = "xxx"
main(selected_urls)

0 
Title in simplified Chinese:
台湾特斯拉供应商豪田公司正在重新调整其供应链，并将在美国建立首个工厂。

Summary in simplified Chinese:
我会给你提供文字内容。  台湾特斯拉供应商Hota正在重新调整其供应链，以满足美国工厂的需求。Hota表示，他们正在加大对美国市场的投入，以满足特斯拉在美国的生
产需求。这一调整将使Hota可以更好地满足全球市场的需求，并有助于加强与特斯拉的合作关系。Hota表示，他们正在寻求更多资源来支持特斯拉在美国的生产，并将继续努
力满足全球客户的需求。


/Users/wjq/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


1 
Title in simplified Chinese:
特斯拉与丰田的大战：决定汽车未来的巨大对决

Summary in simplified Chinese:
来源  Tesla和Toyota之间将发生一场自动驾驶的未来之战，据《华尔街日报》报道，两家公司正在加紧研发自动驾驶技术，争夺市场份额。Tesla正在推出由自动
驾驶软件驱动的全新汽车，而Toyota则将继续投入大量资金，用于开发更加安全可靠的自动驾驶系统。两家公司的竞争将对未来汽车行业产生重大影响，并且可能会改变消费者
的购买习惯。
2 
Title in simplified Chinese:
Sportage仍然位居榜首，这是全新的Picanto：“更好的外观，理想的家庭车”

Summary in simplified Chinese:
我会尝试帮你翻译。  近日，韩国汽车制造商Kia发布了其新款车型Sportage的最新消息，该车型仍然是市场上的领头羊。此外，Kia还推出了新款车型Picant
o，它的外观更加精致，是家庭购车的首选。Picanto的内饰和安全性能也有了很大的提升，更加适合家庭使用。Kia公司表示，新款Picanto将会为消费者带来更好
的体验，让他们拥有更安全、更舒适的出行体验。
3 
Title in simplified Chinese:
梅赛德斯EQS SUV是否将来会由不来梅生产？

Summary in simplified Chinese:
我会提供翻译  据报道，梅赛德斯-奔驰（Mercedes-Benz）正计划在其布雷梅恩（Bremen）工厂生产其未来的全电动SUV车型——EQS。梅赛德斯-
奔驰计划在2023年推出EQS，并将其纳入其布雷梅恩工厂的生产线。该工厂目前正在生产梅赛德斯GLC和GLC
Coupe车型，并将在2022年推出梅赛德斯EQA车型。梅赛德斯-
奔驰表示，EQS将是其未来的旗舰SUV，其将采用全新的架构，并将采用全新的技术，以满足未来消费者的需求。
4 
Title in simplified Chinese:
行业领袖呼吁政府提供激励措施来帮助私人购买电动汽车。

Summary in simplified Chinese:
我会帮助你。  这则新闻报道了业界高管呼吁政府提供购买电动汽车的激励措施，以帮助私人购买者。他们认为，政府应

In [15]:
# create the final file for generation
title = pd.read_excel("~/Downloads/日报抬头1.xlsx")
column_names = title.columns.tolist()
selected_urls.columns = column_names
selected_urls.to_excel(f"~/Downloads/EV_CAR - {final_name}.xlsx", encoding='utf-16',index=False)

# ****executive summary here

In [16]:
url_all = ''
for url in selected_urls['链接']:
    url_all = url_all + '\n' + url

KeyError: '链接'

In [219]:
print(f'{url_all}\nYou are a news anchor who needs to quickly broadcast today\'s news about electric car companies. Please summarize all the news from the above {len(selected_urls)} links in a Chinese paragraph of more than 800 characters, with the following three requirements: 1. Do not start new sentences with phrases like \'the first article\', \'the second article\'; all news should naturally connect into one paragraph. 2. The summary needs to cover the overall trends in the electric vehicle industry as well as the brands and events that have received the most media attention. 3. Mention whether the overall sentiment of these news stories is positive or negative')


http://darcyhitchcock.wordpress.com/2023/07/11/porsche-can-go-800-miles-on-a-charge/
https://uk.style.yahoo.com/toyota-targets-europe-china-hydrogen-114111162.html
https://www.phoneweek.co.uk/plans-for-uks-largest-electric-vehicle-rapid-charging-hub-revealed/
https://greenfleet.net/news/11072023/instavolt-announces-plans-ev-rapid-charging-super-hub
https://www.corriere.it/motori/news/anteprime/23_luglio_11/mazda-mx-30-r-ev-arriva-italia-crossover-che-si-trasforma-elettrico-ibrido-plug-in-41da4186-18f5-11ee-9831-52b9a1a1d955.shtml
https://onlineev.com/electromin-to-launch-kingdomss-first-ev-ultra-fast-dc-charging-arab-news/
https://cieonline.co.uk/molex-secures-large-scale-series-production-from-bmw-group-for-next-gen-electric-vehicle-class/
https://globalrubbermarkets.com/2023/07/11/china-gallium-curbs-raise-chip-questions-for-future-ev-models/
https://evmagz.com/zeekr-to-launch-two-electric-vehicle-models-in-israel/
https://motor-news.space/2023/car-reviews/2024-mercedes-amg-eqe-suv-

----------

#### Event cluster analyses

In [17]:
events = pd.read_csv("~/Downloads/230907raw.csv", sep=",")

In [18]:
events['count'] = events.groupby('hit_sentence_zh_clusterId')['hit_sentence'].transform('count')

In [19]:
events = events.sort_values(by = 'count', ascending = False)

In [21]:
# events.sort_values(by = 'count', ascending = False).groupby('count').head(5)

In [69]:
# values = [67.0, 181.0, 135.0, 122.0, 92.0, 86.0, 114.0]  # Replace with your five different values

# # Select rows where 'count' is in 'values'
# events = events[events['count'].isin(values)]

In [22]:
# first five
events.loc[events['count'] == 1463.0, 'hit_sentence_zh_event'] = 'Model S 使用 Tesla 最新的自动驾驶软件 FSD v12 进行自动驾驶'
events.loc[events['count'] == 677.0, 'hit_sentence_zh_event'] = '在售出 61,575 辆电动汽车并损失 45 亿美元后，福特首席执行官承认这些电动汽车很糟糕'
events.loc[events['count'] == 385.0, 'hit_sentence_zh_event'] = 'EVgo 和亚马逊通过 Alexa 推出无缝电动汽车充电器导航体验'
events.loc[events['count'] == 361.0, 'hit_sentence_zh_event'] = 'Stellantis 与 AGI 签署协议支持美国全国经销商电气化和电动汽车充电能力'

In [23]:
events.loc[events['count'] == 1463.0, 'url'] = 'https://twitter.com/chiragkhutia/status/1695604939199037587'
events.loc[events['count'] == 677.0, 'url'] = 'https://www.zerohedge.com/technology/ford-ceo-admits-reality-check-when-he-took-electric-f-150-truck-road-trip'
events.loc[events['count'] == 385.0, 'url'] = 'https://markets.financialcontent.com/stocks/article/bizwire-2023-8-31-evgo-and-amazon-launch-seamless-ev-charger-navigation-experience-with-alexa'
events.loc[events['count'] == 361.0, 'url'] = 'https://www.ky3.com/prnewswire/2023/08/23/stellantis-completes-agreement-with-agi-support-national-us-dealership-electrification-ev-charging-capabilities/'

In [24]:
# concat the new df and save to excel
counts = [1463.0, 677.0, 385.0, 361.0]

# Create an empty DataFrame to store the results
selected_events = pd.DataFrame()

# Loop over the counts and append the selected rows to the new DataFrame
for count in counts:
    selected_rows = events[events['count'] == count]
    selected_events = pd.concat([selected_events, selected_rows])
    
selected_events = selected_events[['ID','hit_sentence_zh_event','url']]
selected_events.to_excel(f"~/Downloads/EV_event - {final_name}.xlsx", encoding="UTF-16", index=False)

In [25]:
# check the final output
check_events = pd.read_excel("~/Downloads/EV_event - 2023-09-13.xlsx")
check_events.groupby('hit_sentence_zh_event')['ID'].count().sort_values()

hit_sentence_zh_event
Stellantis 与 AGI 签署协议支持美国全国经销商电气化和电动汽车充电能力         361
EVgo 和亚马逊通过 Alexa 推出无缝电动汽车充电器导航体验                  385
在售出 61,575 辆电动汽车并损失 45 亿美元后，福特首席执行官承认这些电动汽车很糟糕     677
Model S 使用 Tesla 最新的自动驾驶软件 FSD v12 进行自动驾驶         1463
Name: ID, dtype: int64